In [1]:
#Description : this program attemps to predicts if the stock of a company will increase or decrease
#              based on top news headlines.

#Import the python libraries

from pandas_datareader import data as web
import pandas as pd
import numpy as np
from datetime import datetime
from textblob import TextBlob
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
#Get the stock symbols/ tickers in the portfolio

#FACEBOOK

assets = ['AMZN']




In [2]:
pip install vaderSentiment

     |████████████████████████████████| 125 kB 8.3 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install textblob

     |████████████████████████████████| 636 kB 279 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Assign weights to the stocks.
weights = np.array([1.0])

In [4]:
#Get the stock/portfolio starting date 
stockStartDate = '2013-01-01'

In [5]:
#Get the stocks ending date (today)
today = datetime.today().strftime('%Y-%m-%d')
today

'2021-04-06'

In [115]:
#Create a dataframe to store the adjusted close price of the stock
df1 = pd.DataFrame()

#Store the adjusted close price of the stock into the df
for  stock in assets:
    df1[stock] = web.DataReader(ticker, data_source='yahoo', start = stockStartDate, end = today)['Business']

KeyError: 'Business'

In [80]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# NLTK VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer

finwiz_url = 'https://finviz.com/quote.ashx?t='

In [81]:
news_tables = {}
tickers = ['AMZN', 'TSLA', 'GOOG']

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    response = urlopen(req)    
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response)
    # Find 'news-table' in the Soup and load it into 'news_table'
    news_table = html.find(id='news-table')
    # Add the table to our dictionary
    news_tables[ticker] = news_table


In [82]:
# Read one single day of headlines for 'AMZN' 
amzn = news_tables['AMZN']
# Get all the table rows tagged in HTML with <tr> into 'amzn_tr'
amzn_tr = amzn.findAll('tr')

for i, table_row in enumerate(amzn_tr):
    # Read the text of the element 'a' into 'link_text'
    a_text = table_row.a.text
    # Read the text of the element 'td' into 'data_text'
    td_text = table_row.td.text
    # Print the contents of 'link_text' and 'data_text' 
    print(a_text)
    print(td_text)
    # Exit after printing 4 rows of data
    if i == 3:
        break

Dow Jones Futures: Stock Market Rally Pauses Amid Biden Tax Hike Plans, But Apple, Tech Titans Lead; Tesla Delays Continue
Apr-07-21 08:49PM  
Is Walmart Stock A Buy Right Now? Here's What Charts, Analysis Show
06:02PM  
Is UPS Stock A Buy Right Now? Here's What Charts, Earnings Show
05:53PM  
Amazon (AMZN) Outpaces Stock Market Gains: What You Should Know
05:45PM  


In [83]:
parsed_news = []

# Iterate through the news
for file_name, news_table in news_tables.items():
    # Iterate through all tr tags in 'news_table'
    for x in news_table.findAll('tr'):
        # read the text from each tr tag into text
        # get text from a only
        text = x.a.get_text() 
        # splite text in the td tag into a list 
        date_scrape = x.td.text.split()
        # if the length of 'date_scrape' is 1, load 'time' as the only element

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        # else load 'date' as the 1st element and 'time' as the second    
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        # Extract the ticker from the file name, get the string up to the 1st '_'  
        ticker = file_name.split('_')[0]
        
        # Append ticker, date, time and headline as a list to the 'parsed_news' list
        parsed_news.append([ticker, date, time, text])
        
parsed_news

[['AMZN',
  'Apr-07-21',
  '08:49PM',
  'Dow Jones Futures: Stock Market Rally Pauses Amid Biden Tax Hike Plans, But Apple, Tech Titans Lead; Tesla Delays Continue'],
 ['AMZN',
  'Apr-07-21',
  '06:02PM',
  "Is Walmart Stock A Buy Right Now? Here's What Charts, Analysis Show"],
 ['AMZN',
  'Apr-07-21',
  '05:53PM',
  "Is UPS Stock A Buy Right Now? Here's What Charts, Earnings Show"],
 ['AMZN',
  'Apr-07-21',
  '05:45PM',
  'Amazon (AMZN) Outpaces Stock Market Gains: What You Should Know'],
 ['AMZN',
  'Apr-07-21',
  '04:06PM',
  'U.S. SEC blocks Amazon effort to stop shareholder votes on racial equity audit'],
 ['AMZN',
  'Apr-07-21',
  '04:00PM',
  'Tencent Bets Billions on Gamers With More Fans Than NBA Stars'],
 ['AMZN',
  'Apr-07-21',
  '03:17PM',
  "Column: Jeff Bezos likes Biden's infrastructure plan because he knows it's worth the money"],
 ['AMZN',
  'Apr-07-21',
  '02:50PM',
  'New robot Stretch to amplify productivity in warehouses: Boston Dynamics CEO'],
 ['AMZN',
  'Apr-07-

In [169]:
# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()

# Set column names
columns = ['ticker', 'date', 'time', 'headline', ]

# Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)

# Iterate through the headlines and get the polarity scores using vader
scores = parsed_and_scored_news['headline'].apply(vader.polarity_scores).tolist()

# Convert the 'scores' list of dicts into a DataFrame
scores_df = pd.DataFrame(scores)

# Join the DataFrames of the news and the list of dicts
parsed_and_scored_news = parsed_and_scored_news.join(scores_df, rsuffix='_right')

# Convert the date column from string to datetime
parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date

parsed_and_scored_news.head()

,ticker,date,time,headline,neg,neu,pos,compound
0,AMZN,2021-04-07,08:49PM,Dow Jones Futures: Stock Market Rally Pauses A...,0.000,1.000,0.000,0.0000
1,AMZN,2021-04-07,06:02PM,Is Walmart Stock A Buy Right Now? Here's What ...,0.000,1.000,0.000,0.0000
2,AMZN,2021-04-07,05:53PM,Is UPS Stock A Buy Right Now? Here's What Char...,0.000,1.000,0.000,0.0000
3,AMZN,2021-04-07,05:45PM,Amazon (AMZN) Outpaces Stock Market Gains: Wha...,0.000,0.661,0.339,0.4767
4,AMZN,2021-04-07,04:06PM,U.S. SEC blocks Amazon effort to stop sharehol...,0.259,0.633,0.108,-0.3400


In [146]:
# import pandas as pd

# parsed_and_scored_news = pd.DataFrame() 

# # saving the dataframe 
# parsed_and_scored_news.to_csv('AMZNO news headlines.csv')

In [149]:
#Get the number of row  and columns for df2
parsed_and_scored_news.shape

(300, 8)

In [24]:
import nltk
nltk.downloader.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/thembajsph/nltk_data...


True

In [13]:
#Create a dataframe to store the adjusted close price of the stock
df2 = pd.DataFrame()

#Store the adjusted close price of the stock into the df
for  stock in assets:
    df2[stock] = web.DataReader(ticker, data_source='yahoo', start = stockStartDate, end = today)['Adj Close']

In [1]:
pip install yfinance


Note: you may need to restart the kernel to use updated packages.


In [125]:
import pandas as pd

df = pd.DataFrame(data) 

# saving the dataframe 
df.to_csv('AMZNO STOCK.csv')

In [126]:
data.stack().reset_index().rename(index=str, columns={"level_1": "Symbol"}).sort_values(['Symbol','Date'])

,Date,Symbol,0
4,2012-01-03,Adj Close,1.790300e+02
10,2012-01-04,Adj Close,1.775100e+02
16,2012-01-05,Adj Close,1.776100e+02
22,2012-01-06,Adj Close,1.826100e+02
28,2012-01-09,Adj Close,1.785600e+02
...,...,...,...
13943,2021-03-29,Volume,2.746000e+06
13949,2021-03-30,Volume,2.337600e+06
13955,2021-03-31,Volume,3.093900e+06
13961,2021-04-01,Volume,2.940300e+06


In [127]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-01-03,175.889999,179.479996,175.550003,179.029999,179.029999,5110800
2012-01-04,179.210007,180.500000,176.070007,177.509995,177.509995,4205200
2012-01-05,175.940002,178.250000,174.050003,177.610001,177.610001,3809100
2012-01-06,178.070007,184.649994,177.500000,182.610001,182.610001,7008400
2012-01-09,182.759995,184.369995,177.000000,178.559998,178.559998,5056900
...,...,...,...,...,...,...
2021-03-29,3055.439941,3091.250000,3028.449951,3075.729980,3075.729980,2746000
2021-03-30,3070.010010,3073.000000,3034.000000,3055.290039,3055.290039,2337600
2021-03-31,3064.060059,3119.330078,3062.500000,3094.080078,3094.080078,3093900


In [143]:
#Show the first 3 rows of data df1
parsed_and_scored_news.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-01-03,175.889999,179.479996,175.550003,179.029999,179.029999,5110800
2012-01-04,179.210007,180.500000,176.070007,177.509995,177.509995,4205200
2012-01-05,175.940002,178.250000,174.050003,177.610001,177.610001,3809100


In [171]:
import pandas_datareader.data as web
import datetime    

start = datetime.datetime(2021, 4, 3)
end = datetime.datetime(2021, 4, 8)
df = web.DataReader("AMZN", 'yahoo', start, end)

dates =[]
for x in range(len(df)):
    newdate = str(df.index[x])
    newdate = newdate[0:10]
    dates.append(newdate)

df['date'] = dates


In [172]:
df.reset_index(inplace=True,drop=False)

In [173]:
df['Date']


0   2021-04-05
1   2021-04-06
2   2021-04-07
Name: Date, dtype: datetime64[ns]

In [174]:
df

,Date,High,Low,Open,Close,Volume,Adj Close,date
0,2021-04-05,3235.959961,3161.239990,3173.000000,3226.729980,3334900,3226.729980,2021-04-05
1,2021-04-06,3247.310059,3217.040039,3223.750000,3223.820068,2507400,3223.820068,2021-04-06
2,2021-04-07,3303.582520,3223.649902,3233.800049,3279.389893,3221385,3279.389893,2021-04-07


In [175]:
df.head()

,Date,High,Low,Open,Close,Volume,Adj Close,date
0,2021-04-05,3235.959961,3161.239990,3173.000000,3226.729980,3334900,3226.729980,2021-04-05
1,2021-04-06,3247.310059,3217.040039,3223.750000,3223.820068,2507400,3223.820068,2021-04-06
2,2021-04-07,3303.582520,3223.649902,3233.800049,3279.389893,3221385,3279.389893,2021-04-07


In [176]:
df.shape


(3, 8)

In [150]:
#Get the number of rows and columns
parsed_and_scored_news.shape

(300, 8)

In [151]:
parsed_and_scored_news.head()


,ticker,date,time,headline,neg,neu,pos,compound
0,AMZN,2021-04-07,08:49PM,Dow Jones Futures: Stock Market Rally Pauses A...,0.000,1.000,0.000,0.0000
1,AMZN,2021-04-07,06:02PM,Is Walmart Stock A Buy Right Now? Here's What ...,0.000,1.000,0.000,0.0000
2,AMZN,2021-04-07,05:53PM,Is UPS Stock A Buy Right Now? Here's What Char...,0.000,1.000,0.000,0.0000
3,AMZN,2021-04-07,05:45PM,Amazon (AMZN) Outpaces Stock Market Gains: Wha...,0.000,0.661,0.339,0.4767
4,AMZN,2021-04-07,04:06PM,U.S. SEC blocks Amazon effort to stop sharehol...,0.259,0.633,0.108,-0.3400


In [137]:
#Print the first 3 rows of data for df2
df2.head()

,AMZN
Date,
2013-01-02,360.274597
2013-01-03,360.483826
2013-01-04,367.607117
2013-01-07,366.003143
2013-01-08,365.280823


In [138]:
#Get the number of row  and columns for df2
df2.shape

(2078, 1)

In [158]:
# import pandas as pd
# parsed_and_scored_news = pd.read_csv("parsed_and_scored_news.csv")
# df = pd.read_csv("stock_price.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'parsed_and_scored_news.csv'

In [177]:
#Merge the data set on the date field
merge = df.merge(parsed_and_scored_news, how='outer', on='date')

In [139]:
merge = pd.concat(df, parsed_and_scored_news)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [178]:
merge.head()

,Date,High,Low,Open,Close,Volume,Adj Close,date,ticker,time,headline,neg,neu,pos,compound
0,2021-04-05,3235.959961,3161.239990,3173.000000,3226.729980,3334900.0,3226.729980,2021-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-04-06,3247.310059,3217.040039,3223.750000,3223.820068,2507400.0,3223.820068,2021-04-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-04-07,3303.582520,3223.649902,3233.800049,3279.389893,3221385.0,3279.389893,2021-04-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-07,AMZN,08:49PM,Dow Jones Futures: Stock Market Rally Pauses A...,0.0,1.0,0.0,0.0
4,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-07,AMZN,06:02PM,Is Walmart Stock A Buy Right Now? Here's What ...,0.0,1.0,0.0,0.0


In [179]:
merge.shape

(303, 15)

In [119]:
merge = pd.merge(df, parsed_and_scored_news, on="id")


KeyError: 'id'

In [121]:
df_join_no_duplicates = df.set_index('Date').join(parsed_and_scored_news.set_index('date'))

print(df_join_no_duplicates)

KeyError: "None of ['Date'] are in the columns"

In [180]:
#Combine the top news headlines
headlines = []

for row in range(0, len(merge.index)):
    headlines.append(' '.join( str(x) for x in merge.iloc[row, 2:27]) )

In [182]:
#Print  a sample of the combined headlines
headlines[3]

'nan nan nan nan nan 2021-04-07 AMZN 08:49PM Dow Jones Futures: Stock Market Rally Pauses Amid Biden Tax Hike Plans, But Apple, Tech Titans Lead; Tesla Delays Continue 0.0 1.0 0.0 0.0'

In [183]:
#Clean the data
clean_headlines = []

for i in range(0, len(headlines)):
    clean_headlines.append(re.sub("b[(')]", '', headlines[i])) #remove b '
    clean_headlines[i] = re.sub('b["]', '', clean_headlines[i]) #remove b"
    clean_headlines[i] = re.sub("\'", '', clean_headlines[i]) #remove \'

NameError: name 're' is not defined

In [135]:
#Show the combined  cleaned headlines
clean_headlines[20]

IndexError: list index out of range

In [184]:
#Add the clean headlines to the merge data set
merge['Combined_News'] = headlines

#Show the new column
merge['Combined_News'][0]

'3161.239990234375 3173.0 3226.72998046875 3334900.0 3226.72998046875 2021-04-05 nan nan nan nan nan nan nan'

In [187]:
#Show the first 3 rows of the merge data set
merge.head(4)

,Date,High,Low,Open,Close,Volume,Adj Close,date,ticker,time,headline,neg,neu,pos,compound,Combined_News
0,2021-04-05,3235.959961,3161.239990,3173.000000,3226.729980,3334900.0,3226.729980,2021-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3161.239990234375 3173.0 3226.72998046875 3334...
1,2021-04-06,3247.310059,3217.040039,3223.750000,3223.820068,2507400.0,3223.820068,2021-04-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3217.0400390625 3223.75 3223.820068359375 2507...
2,2021-04-07,3303.582520,3223.649902,3233.800049,3279.389893,3221385.0,3279.389893,2021-04-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3223.64990234375 3233.800048828125 3279.389892...
3,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-07,AMZN,08:49PM,Dow Jones Futures: Stock Market Rally Pauses A...,0.0,1.0,0.0,0.0,nan nan nan nan nan 2021-04-07 AMZN 08:49PM Do...


In [192]:
from textblob import TextBlob



#Create a function  to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity


In [193]:
#Create two new columns 'subjectivity' and 'Polarity'
merge['Subjectivity'] = merge['Combined_News'].apply(getSubjectivity)
merge['Polarity'] = merge['Combined_News'].apply(getPolarity)

In [98]:
#Create two new columns 'subjectivity' and 'Polarity'
merge['Subjectivity'] = merge['Combined_News'].apply(getSubjectivity)
merge['Polarity'] = merge['Combined_News'].apply(getPolarity)

In [195]:
#Show the new columns in the merge data set
merge.head(4)

,Date,High,Low,Open,Close,Volume,Adj Close,date,ticker,time,headline,neg,neu,pos,compound,Combined_News,Subjectivity,Polarity
0,2021-04-05,3235.959961,3161.239990,3173.000000,3226.729980,3334900.0,3226.729980,2021-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3161.239990234375 3173.0 3226.72998046875 3334...,0.0,0.0
1,2021-04-06,3247.310059,3217.040039,3223.750000,3223.820068,2507400.0,3223.820068,2021-04-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3217.0400390625 3223.75 3223.820068359375 2507...,0.0,0.0
2,2021-04-07,3303.582520,3223.649902,3233.800049,3279.389893,3221385.0,3279.389893,2021-04-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3223.64990234375 3233.800048828125 3279.389892...,0.0,0.0
3,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-07,AMZN,08:49PM,Dow Jones Futures: Stock Market Rally Pauses A...,0.0,1.0,0.0,0.0,nan nan nan nan nan 2021-04-07 AMZN 08:49PM Do...,0.0,0.0


In [198]:
#Create a functon to get the sentiment scores
def getSIA(text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    return sentiment


In [219]:
#Get the sentiment scores for each day
compound = []
neg = []
pos = []
neu = []
label = []
SIA = 0

for i in range(0, len(merge['Combined_News'])):
    SIA = getSIA(merge['Combined_News'][i])
    compound.append(SIA['compound'])
    neg.append(SIA['neg'])
    neu.append(SIA['neu'])
    pos.append(SIA['pos'])
    label.append(SIA['pos'])
    

In [220]:
#Store the sentiment scores in the merge data set
merge['Compound'] = compound
merge['Negetive'] = neg
merge['Neutral'] = neu
merge['Positive'] = pos
merge['Label'] = label



In [221]:
#Show the merge data 
merge .head(7)

,Date,High,Low,Open,Close,Volume,Adj Close,date,ticker,time,...,pos,compound,Combined_News,Subjectivity,Polarity,Compound,Negetive,Neutral,Positive,Label
0,2021-04-05,3235.959961,3161.239990,3173.000000,3226.729980,3334900.0,3226.729980,2021-04-05,NaN,NaN,...,NaN,NaN,3161.239990234375 3173.0 3226.72998046875 3334...,0.000000,0.000000,0.0000,0.0,1.00,0.00,0.00
1,2021-04-06,3247.310059,3217.040039,3223.750000,3223.820068,2507400.0,3223.820068,2021-04-06,NaN,NaN,...,NaN,NaN,3217.0400390625 3223.75 3223.820068359375 2507...,0.000000,0.000000,0.0000,0.0,1.00,0.00,0.00
2,2021-04-07,3303.582520,3223.649902,3233.800049,3279.389893,3221385.0,3279.389893,2021-04-07,NaN,NaN,...,NaN,NaN,3223.64990234375 3233.800048828125 3279.389892...,0.000000,0.000000,0.0000,0.0,1.00,0.00,0.00
3,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-07,AMZN,08:49PM,...,0.000,0.0000,nan nan nan nan nan 2021-04-07 AMZN 08:49PM Do...,0.000000,0.000000,0.0000,0.0,1.00,0.00,0.00
4,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-07,AMZN,06:02PM,...,0.000,0.0000,nan nan nan nan nan 2021-04-07 AMZN 06:02PM Is...,0.535714,0.285714,0.0000,0.0,1.00,0.00,0.00
5,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-07,AMZN,05:53PM,...,0.000,0.0000,nan nan nan nan nan 2021-04-07 AMZN 05:53PM Is...,0.535714,0.285714,0.0000,0.0,1.00,0.00,0.00
6,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-07,AMZN,05:45PM,...,0.339,0.4767,nan nan nan nan nan 2021-04-07 AMZN 05:45PM Am...,0.000000,0.000000,0.4767,0.0,0.83,0.17,0.17


In [223]:
#Create a list of columns to keep
keep_columns = ['Open', 'High', 'Low', 'Volume', 'Subjectivity', 'Polarity', 'Compound', 'Negetive', 'Neutral', 'Positive' , 'Label']
df = merge[keep_columns]
df

,Open,High,Low,Volume,Subjectivity,Polarity,Compound,Negetive,Neutral,Positive,Label
0,3173.000000,3235.959961,3161.239990,3334900.0,0.000000,0.000000,0.0000,0.000,1.000,0.000,0.000
1,3223.750000,3247.310059,3217.040039,2507400.0,0.000000,0.000000,0.0000,0.000,1.000,0.000,0.000
2,3233.800049,3303.582520,3223.649902,3221385.0,0.000000,0.000000,0.0000,0.000,1.000,0.000,0.000
3,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0000,0.000,1.000,0.000,0.000
4,NaN,NaN,NaN,NaN,0.535714,0.285714,0.0000,0.000,1.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...
298,NaN,NaN,NaN,NaN,0.750000,0.250000,-0.1027,0.069,0.931,0.000,0.000
299,NaN,NaN,NaN,NaN,0.344444,0.322222,0.5859,0.000,0.868,0.132,0.132
300,NaN,NaN,NaN,NaN,0.500000,0.500000,0.4939,0.056,0.760,0.184,0.184
301,NaN,NaN,NaN,NaN,0.250000,0.250000,0.2023,0.000,0.924,0.076,0.076


In [224]:
#Create the feature data set
x = df
x = np.array(x.drop(['Label'], 1))
#Create the target data set
y = np.array(df['Label'])

#Split the data int 80% training and 20 testing data sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

NameError: name 'np' is not defined

In [217]:
#Create and train the model
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

NameError: name 'LinearDiscriminantAnalysis' is not defined

In [216]:
#Show the models predictions
predictions = model.predict(x_test)
predictions

NameError: name 'model' is not defined

In [ ]:
y_test

In [ ]:
#Show the model metrics
print( classification_report(y_test, predictions))